In [3]:
from torch.nn import Conv1d
import torch

In [462]:
conv_dim=8
conv_bias = False
d_conv = 8
conv1d = Conv1d(
            in_channels=conv_dim,
            out_channels=conv_dim,
            bias=conv_bias,
            kernel_size=d_conv,
            groups=conv_dim,
            padding=d_conv - 1,
            #**factory_kwargs,
        )

In [463]:
conv1d

Conv1d(8, 8, kernel_size=(8,), stride=(1,), padding=(7,), groups=8, bias=False)

In [480]:
size = 1024
assert size%8 ==0
data = torch.randn(4,conv_dim,size)
whole_res = conv1d(data)

In [481]:
whole_res.shape
#Total output size is d_conv-1 = (2*padding -(kernel -1))

torch.Size([4, 8, 1031])

In [482]:
from einops import rearrange
s = size//8 #subsequence length
padding = d_conv - 1
print(padding)
data_split = [data[:,:,max(0,s*i):s*(i+1)+padding] for i in range(data.shape[-1]//s)]
#data_split = rearrange(data, 'i j (n k) -> n i j k', n = 2)
#print(1*s+conv_dim)
#print(data_split[1].shape)
for d in data_split:
    print(d.shape)

7
torch.Size([4, 8, 135])
torch.Size([4, 8, 135])
torch.Size([4, 8, 135])
torch.Size([4, 8, 135])
torch.Size([4, 8, 135])
torch.Size([4, 8, 135])
torch.Size([4, 8, 135])
torch.Size([4, 8, 128])


In [483]:
res = [conv1d(x) for x in data_split]
for r in res:
    print(r.shape)
torch.cat(res,dim=-1).shape[-1]-whole_res.shape[-1]

torch.Size([4, 8, 142])
torch.Size([4, 8, 142])
torch.Size([4, 8, 142])
torch.Size([4, 8, 142])
torch.Size([4, 8, 142])
torch.Size([4, 8, 142])
torch.Size([4, 8, 142])
torch.Size([4, 8, 135])


98

In [484]:
#print(res[-1].sum(dim=(0,1))[:-3]-whole_res.sum(dim=(0,1))[:19])
l_b, r_b = d_conv -1, -(d_conv-1)
s_p = s
print(res[0].sum(dim=(0,1))[0:r_b])
print(whole_res.sum(dim=(0,1))[:s+d_conv-1])
print(res[1].sum(dim=(0,1))[l_b:r_b])#[l_b:r_b])
print(whole_res.sum(dim=(0,1))[s+d_conv-1:s*2+d_conv-1])
print(res[2].sum(dim=(0,1))[l_b:r_b])
print(whole_res.sum(dim=(0,1))[s*2+d_conv-1:s*3+d_conv-1])

tensor([  0.6091,   1.5338,   3.0116,   1.6812,   5.2154,   2.9534,   2.6096,
          2.0200,  -1.1160,  -1.4831,  -0.7726,  -0.4137,  -3.5267,   0.4202,
         -5.7768,  -0.3872,   0.4537,  -1.2695,  -2.0984,   0.3771,  -1.5114,
          1.2686,   1.5624,   1.7864,   3.7400,  -1.4852,   3.9187,  -2.3934,
          0.7080,   5.2669,  -0.0904,  -1.1988,   1.7409,  -0.4490,  -0.5236,
         -2.4334,   5.0603,   1.1493,  -4.1790,   0.5379,  -9.3268,  -2.4407,
         -2.3760,  -7.7185,  -6.8241,   2.1141,  -2.3018,   1.5181,   0.8638,
          3.2186,   3.4284,   6.1050,   1.9847,   6.3326,   9.1529,   3.3170,
          7.7066,   3.6997,   2.6323,   0.1008,  -3.0512,  -0.3031,   5.6420,
          5.1077,  -0.2889,   1.2248,  -5.6695,   4.3524,   5.5425,  -6.0713,
         -9.3048,  -3.8791,  -2.6624,  -5.2890,  -6.5840, -12.8727,  -1.8465,
          1.0117,  -7.3475,  -1.8230,  -1.0675,  -0.9378,   6.0332,  -2.9050,
         -5.4279,  -0.3289,  -7.4438,   1.2123,   0.2799,   3.89

In [485]:
res[0] = res[0][:,:,:r_b]
res[-1] = res[-1][:,:,l_b:]
for i in range(1,len(res)-1):
    res[i] = res[i][:,:,l_b:r_b]

In [486]:
(torch.cat(res,dim=-1)-whole_res).sum(dim=(0,1))

tensor([ 0.0000e+00,  0.0000e+00,  1.1269e-07,  ..., -1.9139e-07,
         0.0000e+00,  0.0000e+00], grad_fn=<SumBackward1>)

In [487]:
(torch.cat(res,dim=-1)-whole_res).sum()

tensor(-6.9360e-06, grad_fn=<SumBackward0>)

In [495]:
seqlens = [170, 65, 100]

xs = [torch.randint(0, 1000, (1, seqlen), device='cpu', dtype=torch.long) for seqlen in seqlens]

    # Reference 1: Forward pass with seq_idx
x = torch.cat(xs, dim=1)
seq_idx = torch.cat([torch.full((ids.shape[1],), i, dtype=torch.int32, device='cpu')
                         for i, ids in enumerate(xs)], dim=0).unsqueeze(0)

In [496]:
seq_idx

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2

In [499]:
list(range(10))[:]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]